#Train Caffe model for validation

In [ ]:
import os
os.chdir('../../')
import sys
import time
sys.path.insert(0, './python')
sys.path.append('/local-scratch/xca64/tmp/caffe-master/python/myFunc')
import caffe
import numpy as np
from pylab import *
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
solver = caffe.SGDSolver('models/color_constancy/gehler_482_solver.prototxt')
solver.net.copy_from('models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')

import tempfile

In [ ]:
def run_solver(solver, niter, disp_interval, test_interval):
    blobs = ('loss', 'acc')
    loss, acc = (np.zeros(niter), np.zeros(niter))
    test_loss = (np.zeros(niter/test_interval)) 
    for it in range(niter):
        solver.step(1)  # run a single SGD step in Caffe
        loss[it] = (solver.net.blobs['loss'].data.copy())
        acc[it] = 0#(solver.net.blobs['loss_ang'].data.copy())
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = 'loss: %.3f'%loss[it]
            print '%3d) %s Angular Erro %.3f' % (it, loss_disp, acc[it])     
            #print(solver.net.blobs['fc8_flickr'].data[1], solver.net.blobs['illu'].data[1])
        # if it % test_interval == 0:
        #     print "%3d) test loss is %.3f"% (it ,solver.test_nets[0].blobs['accuracy'].data.copy())
        #     test_loss[it//test_interval] = solver.test_nets[0].blobs['accuracy'].data.copy()
    # Save the learned weights from both nets.
    weight_dir = tempfile.mkdtemp()
    name = 'firstTry'
    weights = {}
    filename = 'weights.%s.caffemodel' % name
    
    weights[name] = os.path.join(weight_dir, filename)
    solver.net.save(weights[name])
    
    return loss, test_loss, acc, weights

In [ ]:
niters = 3000
loss_1, test_loss, acc_1, weights_1 = run_solver(solver, niters,20, 5000)

#Save caffe Model for validation

In [ ]:
solver.net.save('models/color_constancy/result/gehler_482_3000_iters_cross_validation.caffemodel')
#save trainning loss for writing up
#np.save('models/color_constancy/result/loss_1.npy', loss_1)

#Load validation data and net

In [ ]:
solver = caffe.SGDSolver('models/color_constancy/gehler_482_solver_validation.prototxt')
solver.net.copy_from('models/color_constancy/result/gehler_482_3000_iters_cross_validation.caffemodel')
from colorConstancy import * 

In [ ]:
validationIters = 14 #every time forward a batch size image
validationSetSize = 161 #number of validation images
allAngulareErr = np.array([])
allGroundTruth = np.array([0,0,0])
allImg = np.zeros([1,3,227,227])
for it in range(validationIters):
    solver.net.forward()
    estimatedIllu = solver.net.blobs['fc8_flickr'].data.copy()  #batch size illumination estimation result
    groundtruethIllu = solver.net.blobs['illu'].data.copy()  #batch size illumination estimation result
    #print solver.net.blobs['label'].data.copy() 
    #allGroundTruth = np.vstack((allGroundTruth, estimatedIllu))
    #allImg = np.vstack((allImg, solver.net.blobs['data'].data.copy() ))
    allAngulareErr = np.hstack((allAngulareErr, np.squeeze( multiangle(estimatedIllu, groundtruethIllu))))
    print np.shape(allAngulareErr)
    #allAngulareErr = np.vstack((allAngulareErr, multiangle(estimatedIllu, groundtruethIllu)))

In [ ]:
print np.shape(allAngulareErr)

#Compute the 10th, median, average, 75th, 90th percitile

In [ ]:
print allAngulareErr

In [ ]:
print np.percentile(allAngulareErr,10)
print np.median(allAngulareErr)
print np.mean(allAngulareErr)
print np.percentile(allAngulareErr,75)
print np.percentile(allAngulareErr,90)
print np.amin(allAngulareErr)
print np.amax(allAngulareErr)

In [ ]:
temp_result = []
temp_result.append()
final_result = open('/home/xca64/remote/data/final_result.txt', 'a')
for i in range(len(temp_result)):
    f.write(temp_result[i])
f.close()